## **Supreme Court Transcripts Database Design**

### Contents:
 1. Finding Justices Present
 2. Date, Year
 3. Appearances
 4. Sentiment Analysis 
 5. Building the DataFrame
 
## Note to Everyone: Make sure you've uploaded the textfiles from this folder: https://drive.google.com/drive/folders/1aepNIVRUS0rwu-m_fqK7KnWEqhyJFBvW my directory might be different from yours so make sure to check !

In [101]:
import pandas as pd
import regex as re
import numpy as np
import os
from os import listdir
from os.path import isfile,join

# Read in a plain text file
files = []
for i in os.listdir(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles"):
    if i.endswith('.txt'):
        text = open(i,encoding="utf8").read()
        files.append(text)

In [103]:
len(files)

1302

In [104]:
cleaned = []
for txt in files:
    clean = re.sub('\xad','',txt)
    clean = re.sub('\n','',clean)
    clean = re.sub('\\\\','',clean)
    cleaned += [clean]

### **Finding Justices Present**

In [31]:
def unique(lst):
    uni = []
    for i in lst:
        if i not in uni:
            uni += [i]
    return uni

def case_no(txt):
    return unique(re.findall('No.\s*\d+[-]*\d+',txt))

In [133]:
def justices(txt):
    j = re.findall('JUSTICE[A-Z\s]+:',txt)
    justice = sorted(unique(j))
    cleaned_list = [justice[i][:-1] for i in range(len(justice))]
    return cleaned_list

In [35]:
#def justices(texts):
 #   d = {}
  #  for txt in texts:
   #     num = case_no(txt)[0]
    #    j = re.findall('JUSTICE[A-Z\s]+:',txt)
     #   justice = sorted(unique(j))
      #  cleaned_list = [justice[i][:-1] for i in range(len(justice))]
       # d[num] = cleaned_list
    #return d

## **Date, Year**

In [10]:
def date(text):
    return re.findall('\W\w+,*\s+\d+\w*\w*,\s+\d{4}',text)

In [11]:
def year(text):
    return re.findall('\d\d\d\d',date(text))

## **Appearances**

In [36]:
def appearances(text):
    app = re.findall('APPEARANCES:[\s\S]*?Reporting',text)[0]
    app = re.findall('[\s\S]*?;[\s\S]*?\.',app)
    app = [re.sub('\d','',app[i]) for i in range(len(app))]
    remove_appearance = re.sub('APPEARANCES:\s','',app[0])
    app[0] = remove_appearance
    app = [re.sub('[\s\s]+',' ',app[i]) for i in range(len(app))]
    return app

In [37]:
for i in cleaned:
    app = appearances(i)
    for j in app:
        print(j)
    print('-----------------------')

PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners.
 DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents.
 CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents.
-----------------------
 SCOTT A. KELLER, Solicitor General of Texas, Austin, Texas; on behalf of the Appellants.
 EDWIN S. KNEEDLER, Deputy Solicitor General, Department of Justice, Washington, D.C.; on behalf of Appellee United States, in support of the Appellants.
 MAX RENEA HICKS, ESQ., Austin, Texas; on behalf of the Appellees in No.
 -. ALLISON J. RIGGS, ESQ., Durham, North Carolina; on behalf of the Appellees in No.
-----------------------
DAVID L. HORAN, ESQ., Dallas, Texas; on behalf of Petitioner in No.
 -; appointed by this Court.
 JAMES E. RYAN, ESQ., Charlottesville, Virginia; on behalf of Petitioner in No.
-----------------------
 ROBERT C. OWEN, ESQ., Au

IndexError: list index out of range

## **Sentiment Analysis**

**Checkpoint 10/22**

* need to work on finding the regex pattern between speakers
* General pattern: "SPEAKER: anything they say until the next SPEAKER:"

### STEPS:
1. Extract the sentences from each speaker.
2. Develop a function (actually there's one written in the github link below that you could model yours from; it's very good)
3. Test it on various transcripts to ensure it's generalized

### Note to Amal: Please look at these slides for performing sentiment analysis. They are from my IEOR class, and these techniques are very useful. Please let me know if you would like to go over it together.

https://github.com/ikhlaqsidhu/data-x/blob/master/07a-tools-nlp-sentiment_add_missing_si/notebook-nlp-sentiment-analysis-imdb-afo_v2.ipynb

In [30]:
## I was trying to extract the speakers from each text. 

def dialogue(text):
    sents = re.findall('[A-Z\s]+:[\s\S]+?[A-Z]+?:',text) #regex pattern to find all instances
    sents = [re.sub('\d','',i) for i in sents] # cleaning transcript
    sents = [re.sub('[\s\s]+',' ',i) for i in sents] #cleaning transcript
    return sents

other_regex_patters = re.findall('[A-Z\s.]+:[\s\S]+?[A-Z\s]+?:',text)

In [48]:
for i in re.findall('(MR. DABNEY:)([\s\S\]+?)([A-Z][A-Z\s]*:)',cleaned[0]):
    print(i)
    print('')

In [49]:
dialogue(cleaned[0])[1]

" CHIEF JUSTICE ROBERTS: We'll hear argument next in No. -, KSR International versus Teleflex, Incorporated. Mr. Dabney. ORAL ARGUMENT OF JAMES W. DABNEY ON BEHALF OF THE PETITIONER MR. DABNEY:"

# **Building the DataFrame**

#### Justices!
`#print(cleaned[4])`

If we run the above ...

Notice that instead of having the justices' name, it just says "QUESTION". This is usually the case for when it appears there is only one justice present. It seems to be only for Justice Rehnquist.

In [153]:
justice = [justices(i) for i in cleaned]
for i in range(len(justice)):
    justice[i] = ", ".join(justice[i])

In [544]:
for i in range(len(justice)):
    justice[i] = re.sub('JUSTICE','',justice[i])
data['Justices'] = justice

#### Dates!

In [162]:
date(cleaned[0])[0]

' December 1, 2008'

In [164]:
uh = []
for i in cleaned:
    uh += date(i)[0]

In [168]:
hea = []
for i in range(len(cleaned)):
    hea += date(cleaned[i])[0]

In [169]:
len(hea)

21323

#### Year!

In [ ]:
years = []
for i in data['Date']:
    years += re.findall('\d\d\d\d',i)

#### Appearances!

In [377]:
def appearances(text):
    app = re.findall('APPEARANCES:[\s\S]*?Reporting.*?',text)
    if len(app) == 0:
        app = re.findall('APPEARANCES:[\s\S]*?REPORTING.*?',text)
        if len(app) == 0:
            app = re.findall('APPEARANCES:[\s\S]*?C O N T E N T S.*?',text)
            if len(app) == 0:
                app = re.findall('APPEARANCES:[\s\S]*?CONTENTS.*?',text)
    app = app[0]
    app = re.findall('[\s\S]*?;[\s\S]*?\.',app)
    app = [re.sub('\d','',app[i]) for i in range(len(app))]
    remove_appearance = re.sub('APPEARANCES:\s','',app[0])
    app[0] = remove_appearance
    app = [re.sub('[\s\s]+',' ',app[i]) for i in range(len(app))]
    return app

In [391]:
def diff(text):
    app = re.findall('APPEARANCES:*[\s\S]*?Reporting.*?',text)
    if len(app) == 0 or len(app)>400:
        app = re.findall('APPEARANCES:[\s\S]*?REPORTING.*?',text)
        if len(app) == 0:
            app = re.findall('APPEARANCES:[\s\S]*?C O N T E N T S.*?',text)
            if len(app) == 0:
                app = re.findall('APPEARANCES:[\s\S]*?CONTENTS.*?',text)
    app = app[0]
    app = re.sub('\d','',app)
    remove_appearance = re.sub('APPEARANCES:*\s','',app)
    app = re.sub('[\s\s]+',' ',remove_appearance)
    return app

In [417]:
ugh = []
c = 1
for i in cleaned:
    #print(c)
    try:
        ugh+=[diff(i)]
        #print('')
    except:
        ugh+= ["ERROR"]
        continue
    c+=1

In [418]:
for i in ugh:
    if i == "ERROR":
        print(i)

ERROR


In [425]:
#diff(cleaned[1023])

In [421]:
appearances(cleaned[1])

[' SCOTT A. KELLER, Solicitor General of Texas, Austin, Texas; on behalf of the Appellants.',
 ' EDWIN S. KNEEDLER, Deputy Solicitor General, Department of Justice, Washington, D.C.; on behalf of Appellee United States, in support of the Appellants.',
 ' MAX RENEA HICKS, ESQ., Austin, Texas; on behalf of the Appellees in No.',
 ' -. ALLISON J. RIGGS, ESQ., Durham, North Carolina; on behalf of the Appellees in No.']

In [427]:
ugh[1023]

'ERROR'

In [432]:
for i in range(len(ugh)):
    if i == 1023:
        ugh[i] = 'ignore'
    else:
        ugh[i] = re.sub('in No. -','',ugh[i])

In [446]:
reporting_pattern = '[A-Z][a-z]+\sReporting'
REPORTING_pattern = '[A-Z]+\sREPORTING'
contents_pattern = 'CONTENTS'
CONTENTS_pattren = 'C O N T E N T S'

In [445]:
re.sub(re.findall(reporting_pattern,ugh[0])[0],'',ugh[0])

'PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners. DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents. CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents. '

In [470]:
for i in range(len(ugh)):
    if i == 1023:
        ugh[i] == 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'
    else:
        report1 = re.findall(reporting_pattern,ugh[i])
        report2 = re.findall(REPORTING_pattern,ugh[i])
        content1 = re.findall(contents_pattern,ugh[i])
        content2 = re.findall(CONTENTS_pattren,ugh[i])
        if len(report1) > 0:
            ugh[i]=re.sub(report1[0],'',ugh[i])
        elif len(report2) > 0:
            ugh[i]=re.sub(report2[0],'',ugh[i])
        elif len(content1) > 0:
            ugh[i]=re.sub(content1[0],'',ugh[i])
        else:
            ugh[i]=re.sub(content2[0],'',ugh[i])

IndexError: list index out of range

In [468]:
bad = 'JASON D. HAWKINS, ESQ., Assistant Federal Public Defender, Dallas, Texas; for Petitioner. WILLIAM M. JAY, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; for  Respondent, in support of Petitioner. EVAN A. YOUNG, ESQ., Austin, Texas; for amicus curiae, in support of the judgment below; appointed by this Court.'

In [545]:
#ugh

#### Title!

In [105]:
title = [f for f in listdir(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles") if isfile(join(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles", f))]
for i in range(len(title)):
    title[i] = re.sub('.pdf.txt','',title[i])
title.remove('DatabaseDesign1.ipynb')

### ACTUAL DATAFRAME!!!

In [472]:
data = pd.DataFrame({'Title':title,'Case No': cases,'Date':dates,'Year':years,'Justices':justice,'Appearances':ugh})#'Justices':justice,'Appearances':people})

In [548]:
data.to_csv(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\supremecourt\textfiles\data.csv")

In [480]:
data['Appearances'] = data['Appearances'].replace('ignore',bad)

#### People!

In [546]:
people = []
for i in range(len(apps)):
    p = re.findall('[A-Z]+\s[A-Z\.\s]*[A-Z]+',apps[i])
    people += [p]
    
for i in range(len(people)):
    people[i] = ", ".join(people[i])
    
data['People'] = people

In [549]:
data.head()

,Title,Case No,Date,Year,Justices,Appearances,People
0,14 Penn Plaza LLC v. Pyett,No. 07-581,"December 1, 2008",2008,"ALITO, BREYER, GINSBURG, KENNEDY, ROBERTS...","PAUL SALVATORE, ESQ., New York, N.Y.; on behal...","PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. ..."
1,Abbott v. Perez,No. 17-586,"April 24, 2018",2018,"ALITO, BREYER, GINSBURG, GORSUCH, KAGAN, ...","SCOTT A. KELLER, Solicitor General of Texas, ...","SCOTT A. KELLER, EDWIN S. KNEEDLER, MAX RENEA ..."
2,Abbott v. United States,No. 09-479,"October 4, 2010",2010,"ALITO, BREYER, GINSBURG, ROBERTS, SCALIA,...","DAVID L. HORAN, ESQ., Dallas, Texas; on behalf...","DAVID L. HORAN, JAMES E. RYAN"
3,Abdul-Kabir v. Quarterman,No. 05-11284,"January 17, 2006",2006,"ALITO, BREYER, GINSBURG, KENNEDY, ROBERTS...","ROBERT C. OWEN, ESQ., Austin, Tex.; on behalf...","ROBERT C. OWEN, EDWARD L. MARSHALL"
4,Abdur_Rahman v. Bell,No. 01-9094,"November 6, 2002",2002,REHNQUIST REHNQUIST,"JAMES S. LIEBMAN, ESQ., New York, New York; on...","JAMES S. LIEBMAN, PAUL G. SUMMERS, PAUL J. ZID..."
